- 날짜
- 수강생명

In [646]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# step 1. 라이브러리 불러오기

In [647]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import sklearn 

print("pandas version :", pd.__version__)
print("numpy version :", np.__version__)
print("seaborn version :", sns.__version__)
print("matplotlib version :", mpl.__version__)
print("sklearn version :", sklearn.__version__)

In [648]:
DATA_PATH = "/kaggle/input/bike-sharing-demand/"
train = pd.read_csv(DATA_PATH + "train.csv")
test = pd.read_csv(DATA_PATH +"test.csv")
submission = pd.read_csv(DATA_PATH + 'sampleSubmission.csv')
train.shape, test.shape, 

In [649]:
train.info()

In [650]:
test.info()

## Step2. 탐색적 자료분석
- 시각화
    - 날짜 기반
    - train 데이터를 다이렉트로 변화를 주면 전처리를 할때 헷갈림
        - 복제본을 만든다
    - 데이터셋 작음
        - 전체를 쓴다
        - 전체에서 샘플링

In [651]:
temp_df=train.copy()
temp_df.info()

- 시각화를 위한 날짜 데이터 처리 
    - 연도, 월, 일자 ,분, 초

### 시간 데이터 전처리 방법 1
- 퍼포먼스 벤치마킹

In [652]:
temp_df['datetime'][:5]

In [653]:
print(temp_df['datetime'][200].split()[1])
print(temp_df['datetime'][200].split()[1].split(':')[0])

In [654]:
year = temp_df['datetime'][100].split()[0].split('-')[0]
month = temp_df['datetime'][100].split()[0].split('-')[1]
day = temp_df['datetime'][100].split()[0].split('-')[2]
year, month, day

In [655]:
hour = temp_df['datetime'][100].split()[1].split(':')[0]
minutes = time = temp_df['datetime'][100].split()[1].split(':')[1]
seconds = time = temp_df['datetime'][100].split()[1].split(':')[2]
hour, minutes, seconds

In [656]:
import time
import datetime

# 시간테스트
start_time = time.time()

temp_df['date'] = temp_df['datetime'].apply(lambda x: x.split()[0])
temp_df['year'] = temp_df['datetime'].apply(lambda x: x.split()[0].split('-')[0]) 
temp_df['month'] = temp_df['datetime'].apply(lambda x: x.split()[0].split('-')[1]) 
temp_df['day'] = temp_df['datetime'].apply(lambda x: x.split()[0].split('-')[2]) 
temp_df['hour'] = temp_df['datetime'].apply(lambda x: x.split()[1].split(':')[0]) 
temp_df['minutes'] = temp_df['datetime'].apply(lambda x: x.split()[1].split(':')[1]) 
temp_df['seconds'] = temp_df['datetime'].apply(lambda x: x.split()[1].split(':')[2]) 

end_time = time.time()
lambda_ctime = end_time - start_time
print("실행시간 (second) -> ", np.round(lambda_ctime,3))

#temp_df[['year','month','day','hour','minutes','seconds']]

### 시간 데이터 전처리 방법 2
- 이 방법의 시간이 더 단축된다

In [657]:
temp_df['date'] = pd.to_datetime(temp_df['datetime'])
temp_df['year'] = temp_df['date'].dt.year
temp_df['year']

In [658]:
# 시간테스트
start_time = time.time()

temp_df['date'] = pd.to_datetime(temp_df['datetime'])
temp_df['year'] = temp_df['date'].dt.year
temp_df['month'] = temp_df['date'].dt.month
temp_df['day'] = temp_df['date'].dt.day
temp_df['hour'] = temp_df['date'].dt.hour
end_time = time.time()

dt_ctime = end_time - start_time
print("실행시간 (second) -> ", np.round(dt_ctime,3))

#temp_df[['year','month','day','hour']]

In [659]:
temp_df['date']

- 요일 추출

In [660]:
temp_df['weekday'] = temp_df['date'].dt.day_name()
temp_df['weekday']

In [661]:
# 딕셔너리 형태
temp_df['season'] = temp_df['season'].map({
    1:'Spring',
    2:'Summer',
    3:'Fall',
    4:'Winter'
})
temp_df['season']

In [662]:
temp_df['weather'] = temp_df['weather'].map({
    1 : 'Clear',
    2 : 'Mist',
    3 : 'Light Snow,Rain',
    4 : 'Heavy Snow,Rain'
})
temp_df['weather']

In [663]:
temp_df.drop(['minutes'], axis = 1)
temp_df.drop(['seconds'], axis = 1)

## Step 4. 데이터시각화
- 수치를 예측하는 대회
- 종속변수를 시각화
- 분포 확인 후 로그변환을 줄지 안줄지 결정 

In [664]:
fig, ax = plt.subplots(nrows = 1, ncols = 2)


sns.histplot(train['count'] ,ax = ax[0])
sns.histplot(np.log(train['count']) ,ax = ax[1])

# 옵션제목
ax[0].set_title('Normal Graph')
ax[1].set_title('log Graph')

plt.show()

- 막대그래프
    - year, count
    - mount, count
    - day,count
    - hour, count
    - 타이틀

In [665]:
fig, ax = plt.subplots(nrows = 2, ncols = 2)

## 전체 그래프 기본설정
# 막대사이의 간격 
fig.tight_layout()

#  전체 그래프 사이즈 관리
fig.set_size_inches(12,8)

## 각 개별 그래프 입력
sns.barplot(x = 'year', y = 'count', data = temp_df, ax=ax[0,0])
sns.barplot(x = 'month',y = 'count', data = temp_df, ax=ax[0,1])
sns.barplot(x = 'day', y = 'count', data = temp_df, ax=ax[1,0])
sns.barplot(x = 'hour', y = 'count', data = temp_df, ax=ax[1,1])

# 디테일한 옵션
ax[0, 0].set_title("Rental Amounts by Year")
ax[0, 1].set_title("Rental Amounts by month")
ax[1, 0].set_title("Rental Amounts by day")
ax[1, 1].set_title("Rental Amounts by hour")

ax[0,0].tick_params(axis='x',labelrotation=90)
plt.show()

- boxplot
    - season,count
    - weather, count
    - holiday, count
    - workingday, count 

In [666]:
fig, ax = plt.subplots(nrows = 2, ncols = 2)

## 전체 그래프 기본설정
# 막대사이의 간격 
fig.tight_layout()

#  전체 그래프 사이즈 관리
fig.set_size_inches(12,8)

## 각 개별 그래프 입력
sns.boxplot(x = 'season', y = 'count', data = temp_df, ax=ax[0,0])
sns.boxplot(x = 'weather',y = 'count', data = temp_df, ax=ax[0,1])
sns.boxplot(x = 'holiday', y = 'count', data = temp_df, ax=ax[1,0])
sns.boxplot(x = 'workingday', y = 'count', data = temp_df, ax=ax[1,1])

# 디테일한 옵션
ax[0, 0].set_title("Rental Amounts by season")
ax[0, 1].set_title("Rental Amounts by weather")
ax[1, 0].set_title("Rental Amounts by holiday")
ax[1, 1].set_title("Rental Amounts by walkingday")

ax[0,1].tick_params(axis='x',labelrotation=10)
plt.show()

- 포인트플랏
    - 5개 행 그래프
        - hue = workingday, holiday, weekday, season, weather
    - 5개 그래프를 한 이미지로 그리시오

In [667]:
fig, ax = plt.subplots(nrows = 5)

## 전체 그래프 기본설정
#  전체 그래프 사이즈 관리
fig.set_size_inches(12,18)

## 각 개별 그래프 입력
sns.pointplot(x = 'hour', y ='count', hue = 'workingday', data = temp_df, ax=ax[0])
sns.pointplot(x = 'hour', y ='count', hue = 'holiday', data = temp_df, ax=ax[1])
sns.pointplot(x = 'hour', y ='count', hue = 'weekday', data = temp_df, ax=ax[2])
sns.pointplot(x = 'hour', y ='count', hue = 'season', data = temp_df, ax=ax[3])
sns.pointplot(x = 'hour', y ='count', hue = 'weather', data = temp_df, ax=ax[4])

# 디테일한 옵션
ax[3].set_title("Rental Amounts by season")
ax[4].set_title("Rental Amounts by weather")
ax[1].set_title("Rental Amounts by holiday")
ax[0].set_title("Rental Amounts by walkingday")
ax[2].set_title("Rental Amounts by weekday")

plt.show()

- 회귀선을 포함한 산점도 그래프
- 총 4개의 그래프

In [668]:
test.info()

In [669]:
fig, ax = plt.subplots(nrows = 2, ncols = 2)

fig.tight_layout()
fig.set_size_inches(12,10)

sns.regplot(x='temp', y='count', data = temp_df, scatter_kws = {'alpha':0.2}, line_kws={'color':'blue'}, ax=ax[0,0])
sns.regplot(x='atemp', y='count', data = temp_df, scatter_kws = {'alpha':0.2}, line_kws={'color':'blue'}, ax=ax[0,1])
sns.regplot(x='humidity', y='count', data = temp_df, scatter_kws = {'alpha':0.2}, line_kws={'color':'blue'}, ax=ax[1,0])
sns.regplot(x='windspeed', y='count', data = temp_df, scatter_kws = {'alpha':0.2}, line_kws={'color':'blue'}, ax=ax[1,1])

plt.show()

- 히트맵 그래프 그리기
- 상관계수
    - 0~0.2 : 두 변수사이의 상관관계는 없다
    - 0.2~1 : 값이 커질수록 두 변수간 상관관계는 크다

In [670]:
corrMat = temp_df[['temp', 'atemp', 'humidity','windspeed','count']].corr()
corrMat

In [671]:
sns.heatmap(corrMat, annot = True)

## step 3. 데이터 전처리 
- 1. train데이터의 causl, registered 제거
- 2. 날짜데이터 처리
- 3. season 컬럼 처리 필요(숫자를 문자로 바꾸자)
    - 인코딩 변환
- 4. weather 컬럼 처리 필요(숫자를 문자로 바꾸자)
    - 인코딩 변환
- 5. month, day 삭제
    - month는 season이랑 비슷
    - day는 holiday와 working와 비슷
- 6. weather=4인 데이터는 삭제(이상치)
- 7. windspeed 컬럼 삭제(0의 값이 많은데 이유를 몰라서)
- 8. temp, atemp 중 하나 삭제(옵션)
- 9. 모든 숫자를 인코딩(원-핫 인코딩)

### 1. train데이터의 causl, registered 제거

In [672]:
train = train.drop(['casual'], axis = 1)
train = train.drop(['registered'], axis = 1)
train = train.drop(['windspeed'], axis = 1)

In [673]:
train.shape, test.shape

## weather 컬럼지우기
- 4인 데이터는 삭제

In [674]:
train = train[train['weather'] != 4].reset_index(drop=True)
train.shape

### 데이터 합치기
- train, test 데이터합치기

In [675]:
all_data = pd.concat([train, test], ignore_index= True)
all_data.info()

### 2, 5. 날짜 데이터 처리 

In [676]:
all_data['date']=pd.to_datetime(all_data['datetime'])
all_data['year']=all_data['date'].dt.year
all_data['hour']=all_data['date'].dt.hour
all_data['weekday'] = all_data['date'].dt.day_name()

all_data.shape

### 3. season 컬럼 처리 

In [677]:
all_data['season'] = all_data['season'].map({
    1:'Spring',
    2:'Summer',
    3:'Fall',
    4:'Winter'
})
all_data.shape

In [678]:
all_data['season']

### 4. weather 컬럼처리

In [679]:
all_data['weather'] = all_data['weather'].map({
    1 : 'Clear',
    2 : 'Mist',
    3 : 'Light Snow,Rain',
    4 : 'Heavy Snow,Rain'
})

In [680]:
all_data.info()

### 6. 이상치삭제 
- weather = 4 인 데이터 삭제

In [681]:
all_data['weather']

- all_data에서 지우는것이 아니라 train데이터에서만 지워야함

In [682]:
#all_data = all_data[all_data['weather'] != 'Heavy Snow,Rain']
#all_data.shape

### 7. windspeed 컬럼 삭제

In [683]:
all_data = all_data.drop(['windspeed'], axis = 1)
all_data.shape

### date 컬럼 제거
- 년도, 요일, 시간을 뽑아내서 필요가 없다.

In [685]:
all_data = all_data.drop(['datetime','date'], axis = 1)
all_data.shape

### 원-핫 인코딩 

In [686]:
all_data = pd.get_dummies(all_data).reset_index(drop=True)
all_data.shape

In [687]:
all_data.info()

## 데이터셋 나누기
- 훈련데이터와 테스트 데이터로 나누기
- count = 타깃데이터 = 종속데이터
    - 타깃데이터가 있으면 훈련데이터
    - 타깃데이터가 없으면 테스트데이터

In [688]:
train = all_data[~pd.isnull(all_data['count'])] 
# ~ = not(!)
test = all_data[pd.isnull(all_data['count'])]

train.shape, test.shape

### count 칼럼을 제거
- 타깃 데이터만 y로 추출
    - train, test count 컬럼제거

In [689]:
# 타깃값
y = train['count']

train = train.drop(['count'], axis = 1)
test = test.drop(['count'], axis = 1)

train.shape, test.shape

## 모델 훈련 
- LinearRegression 모형 학습

In [690]:
from sklearn.linear_model import LinearRegression
# train_test_split
# 하이퍼파라미터 튜닝
# 은 각자 알아서 하기

lr_model = LinearRegression()

# 모형 학습전 로그변환
log_y = np.log(y)
lr_model.fit(train,log_y)

# 모형 예측
lr_preds = lr_model.predict(test)
lr_preds[:10]

## 모형 예측

In [691]:
# 지수 변환
final_preds = np.exp(lr_preds)
final_preds

## 제출하기

In [692]:
submission['count'] = final_preds 
submission.to_csv('submission.csv', index=False)

In [693]:
submission.head()